In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#IMPORTING LIBARIES
import tensorflow as tf 
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [1]:
#train_data is used for feature scaling and image augmentation (image augmentation is applied to avoid overfitting).
train_data = ImageDataGenerator(rescale = 1./255,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

#defining training set, here size of image is reduced to 64x64, batch of images is kept as 64 and class is defined as 'binary'.
training_set = train_data.flow_from_directory('../input/cat-and-dog/training_set/training_set', batch_size = 64, target_size = (64,64), class_mode = 'binary')

In [1]:
#applying same scale as training set, but only feature scaling is applied. image augmentation is avoided to prevent leakage of testing data.
test_data = ImageDataGenerator(rescale = 1./255)

#defining testing set
testing_set = test_data.flow_from_directory('../input/cat-and-dog/test_set/test_set', batch_size = 64, target_size = (64,64), class_mode = 'binary')

In [1]:
#defining the CNN as a sequence of layers.
cnn = tf.keras.models.Sequential()

CNN IS DIVIDED INTO 4 STEPS

* CONVOLUTION

* POOLING

* FLATTENING

* FULL CONNECTION

# # **STEP - 1) ADDING CONVOLUTIONAL LAYER**

In [1]:
#adding 1st Convolutional layer
#note that in image augmentation we kept the image size as 64x64, therefore input_shape should also be same [64,64,3] (here 3 signifies that this is a colorful image (R,G,B))
cnn.add(tf.keras.layers.Conv2D(filters = 32,kernel_size = 3, input_shape = [64,64,3],activation = 'relu'))
#activation function relu is applied to decrease any linearity that might have arrised while applying filters.

# **STEP - 2) APPLYING MAX POOLING**

Here we will apply the max pooling.Max pooling is a pooling operation that selects the maximum element from the region of the feature map covered by the filter. Thus, the output after max-pooling layer would be a feature map containing the most prominent features of the previous feature map.

In [1]:
# applying max pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

# **Addition of 2nd convolution layer.**

In [1]:
#adding 2nd Convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters = 32,kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

In [1]:
#adding 3rd Convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters = 32,kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

# **STEP -3 ) FLATTENING**

In [1]:
#the input of step 4 is an flattened array,
cnn.add(tf.keras.layers.Flatten())

# **STEP - 4 ) FULL CONNECTION**

In [1]:
#forming an ann with 128 input neurons
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

In [1]:
#adding ouput layer of the ann
cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

In [1]:
#compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [1]:
#training the model
cnn.fit(x = training_set, validation_data = testing_set, epochs = 25)

In [1]:
cnn.save('catdog_cnn_model.h5')

In [1]:
from keras.models import load_model 
classifier = load_model('catdog_cnn_model.h5')

# **PREDICTING VALUES**

In [1]:
import numpy as np
from keras.preprocessing import image
training_set.class_indices

# **0 MEANS CATS AND 1 MEANS DOGS**

In [1]:
image.load_img('../input/cat-and-dog/test_set/test_set/cats/cat.4014.jpg')

In [1]:
#importing images
test_img = image.load_img('../input/cat-and-dog/test_set/test_set/cats/cat.4014.jpg',target_size = (64,64))
#converting image to array
test_img = image.img_to_array(test_img)
test_img = np.expand_dims(test_img,axis = 0)
result = classifier.predict(test_img)
if result[0][0] >= 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)

In [1]:
image.load_img('../input/cat-and-dog/test_set/test_set/dogs/dog.4014.jpg')

In [1]:
#importing images
test_img = image.load_img('../input/cat-and-dog/test_set/test_set/dogs/dog.4014.jpg',target_size = (64,64))
#converting image to array
test_img = image.img_to_array(test_img)
test_img = np.expand_dims(test_img,axis = 0)
result = classifier.predict(test_img)
if result[0][0] >= 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)